<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL-DP-PSO-SGD/FL_PSO_SGD_MNIST_using_batches_Clients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pega o erro de cada um e compara. A partícula com menor erro pede o gbest.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import csv
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math

In [2]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=128, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [3]:
# Definições dos hiperparâmetros
NUM_CLIENTS = 5
NUM_PARTICLES = 20
NUM_RODADAS = 10
INERCIA, C1, C2 = 0.9, 0.8, 0.9
EPSILON = 6/math.sqrt(10)
DELTA = 1e-5
SENSITIVITY = 3
MAX_VELOCITY_NORM = 1.5
BATCH_SIZE = 240
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o model global
global_model= MLP(DEVICE, hidden_size=256)
criterio = nn.CrossEntropyLoss()

training on cuda


In [4]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)


def create_subset(dataset, num_clients):
    indices = list(range(len(dataset)))  # Lista de todos os índices
    random.shuffle(indices)  # Embaralha os índices para aleatoriedade

    subset_size = len(indices) // num_clients  # Tamanho de cada subconjunto
    subsets = [Subset(dataset, indices[i * subset_size : (i + 1) * subset_size]) for i in range(num_clients)]

    dataloaders = [
        DataLoader(subset, batch_size=BATCH_SIZE, shuffle=False)
        for subset in subsets
    ]
    return dataloaders


class Particle:
    def __init__(self, particle_id, client_model):
        self.particle_id = particle_id
        self.weights = copy.deepcopy(client_model.state_dict())  # Conjunto de weights da partícula
        self.best_weights = copy.deepcopy(self.weights)  # pbest (best posição da partícula)
        self.best_loss = float('inf')  # best erro alcançado
        self.velocity = {name: torch.zeros_like(param) for name, param in self.weights.items()}  # velocity do PSO
        self.device = client_model.device  # Dispositivo do model

    def update_pso(self, global_best_weights, INERCIA, C1, C2, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM):
        """Atualiza os weights da partícula usando a equação do PSO."""
        for name in self.weights:
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                INERCIA * self.velocity[name] +
                C1 * local_rand * (self.best_weights[name] - self.weights[name]) +
                C2 * global_rand * (global_best_weights[name] - self.weights[name])
            )
            self.weights[name] += self.velocity[name]

    def evaluate_loss(self, client_model, criterio, data):
        """Calcula a perda da partícula no model do client."""
        client_model.load_state_dict(self.weights)  # Aplica os weights da partícula no model do client
        client_model.eval()
        total_loss = 0
        #loader = DataLoader(data, batch_size=64, shuffle=False)

        with torch.no_grad():
            for inputs, labels in data:
                inputs, labels = inputs.to(client_model.device), labels.to(client_model.device)
                outputs = client_model(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(data)


class Client:
    def __init__(self, client_id, global_model, data, num_particles=5):
        self.client_id = client_id
        self.model = copy.deepcopy(global_model)  # Cada client tem seu próprio model
        self.data = data
        self.num_particles = num_particles
        self.particles = []
        self.best_particle = None
        self.initialize_particles(num_particles)

    def initialize_particles(self, num_particles):
        """Cria um conjunto de partículas associadas ao client."""
        self.particles = [Particle(i, self.model) for i in range(num_particles)]

    def train_with_pso(self, INERCIA, C1, C2, global_best_weights, criterio, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM):
        """Treina as partículas usando PSO e atualiza a best partícula local."""

        for particle in self.particles:
            particle.update_pso(global_best_weights, INERCIA, C1, C2, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM)
            erro = particle.evaluate_loss(self.model, criterio, self.data)
            if erro < particle.best_loss:
                particle.best_loss = erro
                particle.best_weights = copy.deepcopy(particle.weights)
        self.select_best_particle()

    def refine_with_adam(self, criterio):
        """Refina os weights da best partícula usando Adam."""
        self.model.load_state_dict(self.best_particle.best_weights)
        otimizador = optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-5)
        #train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)

        self.model.train()
        for _ in range(5):  # 10 épocas de refinamento com Adam
            for inputs, labels in self.data:
                inputs, labels = inputs.to(self.model.device), labels.to(self.model.device)
                otimizador.zero_grad()
                outputs = self.model(inputs)
                loss = criterio(outputs, labels)
                loss.backward()
                otimizador.step()

        # Atualiza os weights da best partícula com os weights refinados pelo Adam
        self.best_particle.weights = copy.deepcopy(self.model.state_dict())

    def select_best_particle(self):
        """Seleciona a best partícula do client."""
        self.best_particle = min(self.particles, key=lambda p: p.best_loss)


def train_federated(global_model, clients, criterio, num_rodadas, INERCIA, C1, C2, testloader, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM):
    """Treina os clients localmente e sincroniza com o servidor central, validando a acurácia."""

    best_weight_global = copy.deepcopy(global_model.state_dict())  # Inicializa com o model global
    best_loss_global = float('inf')

    for rodada in range(num_rodadas):
        resultados_rodada = []

        for client in clients:
            client.train_with_pso(INERCIA, C1, C2, best_weight_global, criterio, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM)  # Treino com PSO
            client.refine_with_adam(criterio)  # Refinamento com Adam
            erro_client = client.best_particle.best_loss  # Obtém o best erro do client
            resultados_rodada.append((client.client_id, erro_client))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        best_client = random.choice(top_3_results)
        best_client_id = best_client[0]
        best_loss_client = best_client[1]

        best_weight_global = copy.deepcopy(clients[best_client_id].best_particle.weights)
        best_loss_global = best_loss_client

        global_model.load_state_dict(best_weight_global)

        test_loss, test_accuracy = evaluate_model(global_model, criterio, testloader)


        print(f"Rodada {rodada+1}/{num_rodadas}: Client {best_client_id} enviou os weights.")
        print(f"Erro Global Atualizado: {best_loss_global:.4f}")
        print(f"Teste -> Perda: {test_loss:.4f}, Acurácia: {test_accuracy:.2f}%\n")

    print("Treinamento Federado Finalizado!")

def evaluate_model(model, criterio, testloader):
    """Avalia o model global no conjunto de teste."""
    model.eval()  # Modo de avaliação

    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples) * 100

    return test_loss, test_accuracy


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_train = mnist_train.targets.numpy()  # Labels

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_test = mnist_test.targets.numpy()  # Labels

# Dividir treino e teste manualmente como no Iris

# Normalizar como no Iris
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets como no Iris
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)
SUBSET_SIZE = 12000

trainloaders = create_subset(trainset, NUM_CLIENTS)

testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

# Criando os clients
clients = [Client(i, global_model, trainloaders[i], NUM_PARTICLES) for i in range(NUM_CLIENTS)]

# Executando o treinamento federado
train_federated(global_model, clients, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY_NORM)

Rodada 1/10: Client 4 enviou os weights.
Erro Global Atualizado: 2.2980
Teste -> Perda: 0.3774, Acurácia: 93.44%

Rodada 2/10: Client 1 enviou os weights.
Erro Global Atualizado: 0.2424
Teste -> Perda: 0.3403, Acurácia: 94.68%

Rodada 3/10: Client 1 enviou os weights.
Erro Global Atualizado: 0.0632
Teste -> Perda: 0.3724, Acurácia: 95.08%

Rodada 4/10: Client 1 enviou os weights.
Erro Global Atualizado: 0.0487
Teste -> Perda: 0.5206, Acurácia: 95.05%

Rodada 5/10: Client 1 enviou os weights.
Erro Global Atualizado: 0.0291
Teste -> Perda: 0.4556, Acurácia: 95.75%

Rodada 6/10: Client 2 enviou os weights.
Erro Global Atualizado: 0.1625
Teste -> Perda: 0.3882, Acurácia: 93.78%

Rodada 7/10: Client 2 enviou os weights.
Erro Global Atualizado: 0.1147
Teste -> Perda: 0.2978, Acurácia: 95.21%

Rodada 8/10: Client 2 enviou os weights.
Erro Global Atualizado: 0.1147
Teste -> Perda: 0.2978, Acurácia: 95.21%

Rodada 9/10: Client 1 enviou os weights.
Erro Global Atualizado: 0.0134
Teste -> Perda: 